In [50]:
import json
import requests
from collections import defaultdict

In [51]:
row_data = open('mayfes/json_data/todai_shoni.json', 'r')

In [52]:
data = json.load(row_data)

In [53]:
len(data)

300

In [54]:
type(data)

list

In [55]:
type(data[0])

dict

In [56]:
data[0].keys()

dict_keys(['title', 'texts'])

In [57]:
titles = []
texts = []
for dic in data:
    titles.extend(dic['title'])
    texts.extend(dic['texts'])

In [58]:
len(titles), len(texts)

(619, 1567)

In [59]:
import re
# from nltk import word_tokenize
# from nltk import WordNetLemmatizer
# from nltk.util import ngrams
from collections import Counter
import MeCab

ptn = re.compile("^[0-9]+$")
# lemmatizer = WordNetLemmatizer()

In [60]:
def load_stop(): #停止語類
    s = []
    f = open('stop.txt', 'r', encoding='utf8')
    lines = f.readlines()
    for l in lines:
        tmp = l.replace('\n', '') # 改行文字がある場合用
        s.append(tmp)
    f.close()
    return s

# stopwords = load_stop()

In [61]:
txt_dic = {} #キー:テキスト本文 値:単語
counter_list = []  #一テキストの単語のCounterが要素として入る。

for text in texts:
    #単語に区切る http://testpy.hatenablog.com/entry/2016/10/04/010000
    tagger = MeCab.Tagger("-Owakati")        
    result = tagger.parse(text)
    tokens = result.split()

    clean_tokens = [w for w in tokens if len(w)>1]

#     #単語の内、停止語類になく、一文字ではなく、数字のみではないものをclean_tokensに入れる
#     clean_tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stopwords and len(w)>1  and ptn.match(w) is None]

    #txt_dicにキーをテキスト本文、値をclean_tokensにして入れる(使用しないが動作確認用)
    txt_dic[text]= clean_tokens

    #一テキストごとのCounterを要素とするリストcounter_listに追加する。最終的にこの中身を足せばよい。
    counter_list.append(Counter(clean_tokens))

In [62]:
#counterを足しあわせる https://qiita.com/hatchinee/items/a904c1f8d732a4686c9d
all_word_counter = Counter()
for counter in counter_list:
    all_word_counter += counter

In [63]:
#頻度の高い上100個の単語を抽出
frequent_words = []
for counter in all_word_counter.most_common(100):
    frequent_words.append(counter[0])

In [64]:
all_word_counter.most_common()[:10]

[('病院', 368),
 ('外科', 322),
 ('ます', 275),
 ('医学部', 252),
 ('医療', 244),
 ('研究', 235),
 ('...', 212),
 ('日本', 192),
 ('医学', 170),
 ('センター', 164)]

In [65]:
#各辞書は単語をキーに、頻度と種類のタプルを保持
noun_dic = defaultdict(int)
verb_dic = defaultdict(int)
adjective_dic = defaultdict(int)
adverb_dic = defaultdict(int)

for text in texts:
    #単語に区切る http://testpy.hatenablog.com/entry/2016/10/04/010000
    tagger = MeCab.Tagger("-Ochasen")        
    result = tagger.parseToNode(text)
    while result:
        try:
            word = result.surface
            if len(word) > 1:
                word_type = result.feature.split(',')[0]
                if word_type == "名詞":
                    noun_dic[word] += 1
                elif word_type == "動詞":
                    verb_dic[word] += 1
                elif word_type == "形容詞":
                    adjective_dic[word] += 1
                elif word_type == "副詞":
                    adverb_dic[word] += 1
                else:
                    pass
        except:
            pass

        result = result.next

#     clean_tokens = [w for w in tokens if len(w)>1]

#     #単語の内、停止語類になく、一文字ではなく、数字のみではないものをclean_tokensに入れる
#     clean_tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stopwords and len(w)>1  and ptn.match(w) is None]

    #txt_dicにキーをテキスト本文、値をclean_tokensにして入れる(使用しないが動作確認用)
#     txt_dic[text]= clean_tokens

    #一テキストごとのCounterを要素とするリストcounter_listに追加する。最終的にこの中身を足せばよい。
#     counter_list.append(Counter(clean_tokens))

In [66]:
with open('mayfes/text_data/todai_shoni_noun.txt','w') as f:
    for k, v in sorted(noun_dic.items(), key=lambda x:x[1], reverse=True):
#         print("({0} , {1})".format(k, v)) # ==> 5 1, 2 3, 1 4の順番．valueで昇順の並び
        f.write("({0} , {1})\n".format(k, v))

In [67]:
with open('mayfes/text_data/todai_shoni_verb.txt','w') as f:
    for k, v in sorted(verb_dic.items(), key=lambda x:x[1], reverse=True):
#         print("({0} , {1})".format(k, v)) # ==> 5 1, 2 3, 1 4の順番．valueで昇順の並び
        f.write("({0} , {1})\n".format(k, v))

In [68]:
with open('mayfes/text_data/todai_shoni_adjec.txt','w') as f:
    for k, v in sorted(adjective_dic.items(), key=lambda x:x[1], reverse=True):
#         print("({0} , {1})".format(k, v)) # ==> 5 1, 2 3, 1 4の順番．valueで昇順の並び
        f.write("({0} , {1})\n".format(k, v))

In [69]:
with open('mayfes/text_data/todai_shoni_adv.txt','w') as f:
    for k, v in sorted(adverb_dic.items(), key=lambda x:x[1], reverse=True):
#         print("({0} , {1})".format(k, v)) # ==> 5 1, 2 3, 1 4の順番．valueで昇順の並び
        f.write("({0} , {1})\n".format(k, v))